In [7]:
import plotly.express as px
import geopandas as gpd
import pandas as pd

df_municipios = pd.read_csv('parahybaCities.csv', engine='python')

df_municipios = df_municipios.drop(columns=['Tecnologia', 'Tipo de compromisso', 'Compromisso', 
                                             'Origem compromisso', 'Operadora com compromisso', 
                                             'Status compromisso', 'Prazo compromisso', 
                                             'Operadora'], axis=1)

df_municipios = df_municipios.drop_duplicates()

df_municipios['% moradores cobertos'] = df_municipios['% moradores cobertos'].str.replace('%', '').str.replace(',', '.').str.strip()
df_municipios['% moradores cobertos'] = pd.to_numeric(df_municipios['% moradores cobertos'], errors='coerce')
df_municipios['% moradores cobertos'] = df_municipios['% moradores cobertos'].fillna(0)

gdf = gpd.read_file("PB_Municipios_Malha/PB_Municipios_2022.shp")

gdf = gdf.merge(df_municipios,)

grafico = px.choropleth_mapbox(
    gdf,
    geojson=gdf.geometry.__geo_interface__,  # Usando geometria diretamente
    locations=gdf.index,  # Usando o índice do GeoDataFrame
    featureidkey="properties.CD_MUN",  # Ajuste para a coluna correta
    color="% moradores cobertos",  # Usando a coluna de porcentagem
    mapbox_style="open_street_map",
    zoom=4,
    center={"lat": -7.239, "lon": -35.881},
    opacity=0.5,
    color_continuous_scale="Viridis"
)

grafico.update_layout(margin={"t": 0, "l": 0, "r": 0, "b": 0})
grafico.show()


KeyError: 'CD_MUN'